# VacationPy
----


In [1]:
# Importing dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

gmaps.configure(api_key=gkey)

## Read WeatherPy results into dataframe

In [2]:
# Creating path object from WeatherPy output csv
csv_path = os.path.join('..','WeatherPy', 'output_data', 'cities.csv')

# Reading csv into dataframe
vac_df = pd.read_csv(csv_path).reset_index(drop=True).drop(['Unnamed: 0'], axis=1)

# Printing sample of dataframe
vac_df.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Wind (mph),Cloudiness %,Time,Date
0,Bulgan,MN,48.81,103.53,63.68,82,5.68,99,11:23:49,2020-08-05
1,Castro,BR,-24.79,-50.01,50.31,93,4.59,0,00:23:50,2020-08-05
2,Vaini,TO,-21.20,-175.20,75.20,69,6.93,20,16:23:50,2020-08-05
3,Kruisfontein,ZA,-34.00,24.73,48.99,82,2.48,100,05:23:50,2020-08-05
4,Mali,GN,12.08,-12.30,58.87,100,3.06,100,03:23:50,2020-08-05


## Humidity Heatmap

In [3]:
# Pulling dataframe columns into variables
locations = vac_df[['Latitude', 'Longitude']]
humidity = vac_df['Humidity %']

In [4]:
# Initializing map
fig = gmaps.figure()

# Creating heatmap layer, coordinates set on locations list, with weighting determined by humidity %
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                              dissipating=False, max_intensity=100, point_radius=2)
# Overlaying heatmap layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Filter cities into new dataframe with my ideal weather conditions

My ideal weather is between 60-70 degrees fahrenheit, very cloudy, with no wind

In [5]:
ideal_df = vac_df.loc[(vac_df['Max Temp (F)'] <= 68) & (vac_df['Max Temp (F)'] >= 60) 
                      & (vac_df['Cloudiness %'] > 95) & (vac_df['Wind (mph)'] < 5), :]

ideal_df

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Wind (mph),Cloudiness %,Time,Date
69,Fortuna,US,40.60,-124.16,60.01,89,3.00,98,20:16:25,2020-08-04
148,Flin Flon,CA,54.77,-101.86,67.57,84,4.79,99,22:24:25,2020-08-04
169,Eureka,US,40.80,-124.16,61.00,92,4.00,96,20:24:30,2020-08-04
298,Taksimo,RU,56.34,114.88,61.74,83,2.59,100,11:24:23,2020-08-05
315,Plastun,RU,44.76,136.29,64.78,85,4.34,100,13:25:05,2020-08-05
325,Yei,SS,4.09,30.68,63.75,93,2.89,100,06:16:16,2020-08-05
384,Torit,SS,4.41,32.57,66.81,86,1.52,100,06:25:21,2020-08-05
504,Zeya,RU,53.75,127.27,63.50,63,3.40,99,12:25:31,2020-08-05


## Creating a map overlay of hotels in my preferred weather conditions

In [6]:
# Assigning Maps Places syntax as a base call variable
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Creating empty hotels list to store Places call results
hotels = []

# Iterating through dataframe and extracting variables to populate call parameters
for index, row in ideal_df.iterrows():
    params = {"location": f"{row['Latitude']}, {row['Longitude']}",
              "radius": 5000,
              "type": 'hotel',
              "key": gkey}
    
    # Error handling for locations without multiple results
    try:
        response = requests.get(base_url, params=params).json()
        hotels.append(response['results'][1]['name'])
    except:
        hotels.append('None')

In [7]:
# Cleaning up hotel name format
hotels = [i.lower() for i in hotels]
hotels = [i.title() for i in hotels]
ideal_df['Hotel Name'] = hotels

# Renaming dataframe variable
hotel_df = ideal_df

/Users/troyramsey/opt/anaconda3/envs/UODataClass/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# Setting up template for info boxes on marker layer
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Populating list with hotel information
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Creating and formatting marker layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='black',
    stroke_color='black', scale=2,
    info_box_content=hotel_info
)

# Overlaying marker layer onto heatmap
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))